# `AA Workshop 5` — Coding Challenge

Complete the tasks below to practice implementing regression modeling from `W5_Regression_Basics.ipynb`.

Guidelines:
- Work in order. Run each cell after editing with Shift+Enter.
- Keep answers short; focus on making things work.
- If a step fails, read the error and fix it.

By the end you will have exercised:
- running linear regressions with `scikit-learn` using a single or multiple features
- visualizing regression outputs
- comparing training error metrics

## Data

For all following tasks, you will work on the same dataset of ridehailing trips from New York City for a single day (August 1st, 2025). The provided dataset is a subset of the data published by the New York City Taxi & Limousine Commission, which is publicly available [here](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page). Load the data (which can be found in the `data`-folder). Then, restrict our sample to trips with a distance of up to 10 miles and a duration of up to 100 minutes. Finally, inspect the data.

In [ ]:
# your code here



## Task 1 - Predicting Ridehailing Fares in NYC

Use `scikit-learn` to run a linear regression to predict the `base_passenger_fare` based on `trip_miles`. Then, generate a scatter plot of the observed and predicted values.

In [ ]:
# your code here



## Task 2 - Distance or Duration?

Which feature is the better predictor of the fare: distance or duration? Compare the mean absolute error on the training set of regression models using `trip_miles` and `trip_minutes` as single predictors, respectively.

In [ ]:
# your code here



## Task 3 - Uber vs. Lyft

Our dataset also contains information on the provider of each trip, namely Uber or Lyft. Can we use this feature to improve our predictions of the passenger fare? Run a regression using both `trip_minutes` and `provider` as features. Calculate the mean absolute error on the training set, and generate a scatter plot including the regression lines for each provider in different colors.

In [ ]:
# your code here

